In [1]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import csv
import stats_arrays
import scipy as sp
import pandas as pd
import pickle
import time

In [2]:
bw.projects.set_current('iw_integration')

## Adding four parameter beta distribution

In [3]:
from numpy import *
from stats_arrays import *
from scipy import stats


class FourParamBetaUncertainty(UncertaintyBase):

    """
4 PARAM BETA DISTRIBUTIONS
The 4 parameters Beta distribution has the probability distribution function:

The :math:`\\alpha` parameter is ``loc``, and :math:`\\beta` is ``shape``. 
By default, the Beta distribution is defined from 0 to 1; the upper bound can be rescaled with the ``maximum`` parameter
and the lower bound with the ``minimum`` parameter.

Wikipedia: `Beta distribution <http://en.wikipedia.org/wiki/Beta_distribution>`_
    """
    id = 13
    description = "Four Parameters Beta uncertainty"

    @classmethod
    def validate(cls, params):
        scale_param=params['maximum']-params['minimum']
        if (params['loc'] > 0).sum() != params.shape[0]:
            raise InvalidParamsError("Real, positive alpha values are" +
                                     " required for Beta uncertainties.")
        if (params['shape'] > 0).sum() != params.shape[0]:
            raise InvalidParamsError("Real, positive beta values are" +
                                     " required for Beta uncertainties.")
        if (scale_param <= 0).sum():
            raise InvalidParamsError("Scale value must be positive or NaN")

    @classmethod
    def random_variables(cls, params, size, seeded_random=None,
                         transform=False):
        scale_param=params['maximum']-params['minimum']
        if not seeded_random:
            seeded_random = random
        scale = scale_param
        scale[isnan(scale)] = 1
        return (params['minimum'].reshape((-1, 1)) + scale.reshape((-1, 1)) * seeded_random.beta(params['loc'], params['shape'], size=(size,params.shape[0])).T)

    @classmethod
    def cdf(cls, params, vector):
        vector = cls.check_2d_inputs(params, vector)
        results = zeros(vector.shape)
        scale_param=params['maximum']-params['minimum']
        scale = scale_param
        scale[isnan(scale)] = 1
        for row in range(params.shape[0]):
            results[row, :] = stats.beta.cdf(vector[row, :],
                                             params['loc'][row], params['shape'][row],
                                             loc=params['minimum'][row],
                                             scale=scale[row])
        return results

    @classmethod
    def ppf(cls, params, percentages):
        percentages = cls.check_2d_inputs(params, percentages)
        results = zeros(percentages.shape)
        scale_param=params['maximum']-params['minimum']
        scale = scale_param
        scale[isnan(scale)] = 1
        for row in range(percentages.shape[0]):
            results[row, :] = stats.beta.ppf(percentages[row, :],
                                             params['loc'][row], params['shape'][row],
                                             loc=params['minimum'][row],
                                             scale=scale[row])
        return results

    @classmethod
    def statistics(cls, params):
        alpha = float(params['loc'])
        beta = float(params['shape'])
        mini = float(params['minimum'])
        maxi = float(params['maximum'])
        # scale = 1 if isnan(params['maximum'])[0] else float(params['maximum'])
        if alpha <= 1 or beta <= 1:
            mode = "Undefined"
        else:
            mode = mini + (maxi-mini) * (alpha - 1) / (alpha + beta - 2)
        return {
            'mean': mini + (maxi-mini) * alpha / (alpha + beta),
            'mode': mode,
            'median': "Not Implemented",
            'lower': mini,
            'upper': maxi
        }

    @classmethod
    def pdf(cls, params, xs=None):
        scale_param=params['maximum']-params['minimum']
        scale = 1 if isnan(scale_param)[0] else float(scale_param)
        if xs is None:
            xs = arange(0, scale, scale / cls.default_number_points_in_pdf)
        ys = stats.beta.pdf(xs, params['loc'], params['shape'],
                            loc=params['minimum'],
                            scale=scale)
        return xs, ys.reshape(ys.shape[1])

In [4]:
uncertainty_choices.add(FourParamBetaUncertainty)
print (FourParamBetaUncertainty in uncertainty_choices)

True


## Creating ecoinvent v33 DB with and without uncertainty

eiv33 with uncertainty

In [ ]:
fpei33 = r"C:\bw2-python\ecospold_ecoinvent_3.3_cut off"
if 'ecoinvent 3.3 cutoff' in bw.databases:
    print("Database has already been imported")
else:
    ei33 = bw.SingleOutputEcospold2Importer(fpei33, 'ecoinvent 3.3 cutoff')
    ei33.apply_strategies()
    ei33.statistics()
    ei33.write_database()

Creating a version without uncertainty

In [ ]:
#create a copy of ei33
fpei33 = r"C:\bw2-python\ecospold_ecoinvent_3.3_cut off"
if 'ecoinvent 3.3 cutoff without uncertainty' in bw.databases:
    print("Database has already been imported")
else:
    ei33_no_uncert = bw.SingleOutputEcospold2Importer(fpei33, 'ecoinvent 3.3 cutoff without uncertainty')
    ei33_no_uncert.apply_strategies()
    ei33_no_uncert.statistics()
    ei33_no_uncert.write_database()

In [ ]:
def remove_uncert_in_act(act):
    for exc in act.exchanges():
        exc['uncertainty type']=0
        exc.save()
    
    act.save()
    
    return act;

In [ ]:
for act in ei33_db_no_uncert:
    remove_uncert_in_act(act)

## Loading ecoinvent v33 DB with and without uncertainty

In [5]:
ei33_db = bw.Database('ecoinvent 3.3 cutoff')
len(ei33_db)

13831

In [6]:
ei33_db_no_uncert = bw.Database('ecoinvent 3.3 cutoff without uncertainty')
len(ei33_db_no_uncert)

13831

## Loading impact categories

In [8]:
IW_with_uncert_beta=[m for m in bw.methods 
                     if 'IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017' in str(m)
                    and 'with uncert' in str(m)
                    and 'ID14' not in str(m)]
IW_with_uncert_beta_wo=[m for m in IW_with_uncert_beta 
                             if 'Land transformation' not in str(m) 
                             and 'Land occupation' not in str(m)
                             and 'Freshwater eutrophication' not in str(m)]
IW_with_uncert_beta

[('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Marine eutrophication, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Terrestrial acidification, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Human Health',
  'Water availability, human health, GLO, with uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatia

## Creating impact categories without uncertainty

In [25]:
#Change a CF value for uncertainty dictionary or other value

def modify_CF_value(CF_tuple, new_CF_value):
    
    lst=list(CF_tuple)
    lst[1]=new_CF_value #value or uncertainty dictionary
    new_CF_tuple=tuple(lst)
    
    return new_CF_tuple;

In [39]:
#Remove CF uncertainty for several impact categories

def remove_uncert_in_impact_cat(ic_name):

    ic=bw.Method(ic_name)
    cf_list=ic.load()
    
    i=0

    for cf in cf_list:

        if type(cf[1])== dict:
            cf=modify_CF_value(cf, cf[1]['amount'])

        cf_list[i]=cf
        i=i+1

    ic.write(cf_list)
        
    return;

In [40]:
#Copy and rename impact category

def copy_rename_impact_cat(old_ic_name, new_ic_name):
    
    old_ic=bw.Method(old_ic_name)
    old_ic.copy(new_ic_name)
    
    return;

In [58]:
list_of_ic_names=IW_with_uncert_beta_to_create_no_uncert
string_to_replace='with uncert'
string_to_use_instead='no uncert'

for ic_name in list_of_ic_names:
    
    #define new ic name. Here we changed only the first name
    new_ic_name=(ic_name[0],ic_name[1],ic_name[2].replace(string_to_replace,string_to_use_instead))
    
    #copy ic
    old_ic_name=ic_name
    copy_rename_impact_cat(old_ic_name, new_ic_name)
    
    #remove CF uncertainty
    remove_uncert_in_impact_cat(new_ic_name)
    

In [9]:
IW_no_uncert=[m for m in bw.methods 
                     if 'IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017' in str(m)
             and 'no uncert' in str(m)]
IW_no_uncert

[('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Land transformation, biodiversity, GLO, no uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Freshwater acidification, GLO, no uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Marine eutrophication, GLO, no uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Human Health',
  'Water availability, human health, GLO, no uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
  'Ecosystem Quality',
  'Terrestrial acidification, GLO, no uncert'),
 ('IMPACTWorld+ - Endpoint - only with spatial variabil

## Calculating MC results for an entire DB

Let's start with a simplified DB

In [10]:
ei33_db_no_uncert_simplified=[ei33_db_no_uncert.random() for i in range(3)]
ei33_db_no_uncert_simplified

['market for electricity, high voltage' (kilowatt hour, IE, None),
 'wood chips production, hardwood, at sawmill' (kilogram, CA-QC, None),
 'market for titanium dioxide' (kilogram, RER, None)]

In [11]:
act_rand=ei33_db.random()
act_rand.key

('ecoinvent 3.3 cutoff', '688ea8334993f14869ee88613af12bc9')

In [12]:
act_codes_simplified_DB=[act['code'] for act in ei33_db_no_uncert_simplified]
ei33_db_with_uncert_simplified=[act for act in ei33_db if True in [act['code']==act_code for act_code in act_codes_simplified_DB]]
ei33_db_with_uncert_simplified

['market for electricity, high voltage' (kilowatt hour, IE, None),
 'market for titanium dioxide' (kilogram, RER, None),
 'wood chips production, hardwood, at sawmill' (kilogram, CA-QC, None)]

Function for Monte with multiple impact categories, 1 activity

In [13]:
#list_methods_name = IW_with_uncert
#fu_tuple = (act,1)
#iterations = 10

def MC_multi_impact(fu_tuple,list_methods_name,iterations):
    
    MC_results_dict={}
    fu_dict={fu_tuple[0]:fu_tuple[1]}
    
    for ic in list_methods_name:
        print(ic)
        MC_results_per_IC = bw.MonteCarloLCA(fu_dict,ic)
        scores = [next(MC_results_per_IC) for _ in range(iterations)]
        MC_results_dict[ic]=scores
    
    return MC_results_dict;

Function for Monte Carlo, one impact category, one activity

In [14]:
def MC_one_IC(imp_cat_name,act,fu,iterations):
    myFirstMonteCarlo = bw.MonteCarloLCA({act:fu},imp_cat_name)
    scores = [next(myFirstMonteCarlo) for _ in range(iterations)]
    return scores;

Function to calculate MC results for an entire DB

In [15]:
def MC_multi_impact_entire_DB(DB,list_methods_name,iterations):
    
    MC_results={}
    
    number_of_activity=0
    
    for act in DB:
        
        number_of_activity=number_of_activity+1
        print(str(number_of_activity)+" - "+str(act))
        
        fu_tuple = (act,1)
        scores=MC_multi_impact(fu_tuple,list_methods_name,iterations)
        MC_results[act.key]=scores
        
    return MC_results;

Test on the simplified DB

In [18]:
#MC simulation with uncertainty LCI=0 and LCIA=1
DB=ei33_db_no_uncert_simplified
list_methods_name=IW_with_uncert_beta
iterations=10

MC_results_eiv33_simplified_it1000_LCI0_LCIA1=MC_multi_impact_entire_DB(DB,list_methods_name,iterations)

type(MC_results_eiv33_simplified_it1000_LCI0_LCIA1),len(MC_results_eiv33_simplified_it1000_LCI0_LCIA1)

1 - 'market for electricity, high voltage' (kilowatt hour, IE, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Marine eutrophication, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Land transformation, biodiversity, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPA

(dict, 3)

In [19]:
MC_results_eiv33_simplified_it1000_LCI0_LCIA1

{('ecoinvent 3.3 cutoff without uncertainty',
  '89a9df4ca87f9f5413a65755dd5ec18e'): {('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
   'Ecosystem Quality',
   'Freshwater acidification, GLO, with uncert'): [0.011086933786306975,
   0.00018948142057429175,
   0.0009661113207695074,
   8.246152799173109e-05,
   0.00914172722044503,
   0.0004308681969762102,
   0.00045584435147277255,
   6.154375120161143e-05,
   0.0005488960951146597,
   0.018140458618896967],
  ('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
   'Ecosystem Quality',
   'Freshwater eutrophication, GLO, with uncert'): [1.1667722266871089e-11,
   0.0015160504210414362,
   1.1667723125289791e-11,
   1.1667722266871089e-11,
   1.1667722266871089e-11,
   1.2443466949215789e-05,
   1.1667722266871089e-11,
   1.1667722266871089e-11,
   1.1667722266871089e-11,
   1.1667722266871089e-11],
  ('IM

In [20]:
#MC simulation with uncertainty LCI=1 and LCIA=0
DB=ei33_db_with_uncert_simplified
list_methods_name=IW_no_uncert
iterations=10

MC_results_eiv33_simplified_it1000_LCI1_LCIA0=MC_multi_impact_entire_DB(DB,list_methods_name,iterations)


1 - 'market for electricity, high voltage' (kilowatt hour, IE, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Land transformation, biodiversity, GLO, no uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, no uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Marine eutrophication, GLO, no uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, no uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, no uncert')
('IMPACTWorld+ -

In [21]:
#MC simulation with uncertainty LCI=1 and LCIA=1
DB=ei33_db_with_uncert_simplified
list_methods_name=IW_with_uncert_beta
iterations=10

MC_results_eiv33_simplified_it1000_LCI1_LCIA1=MC_multi_impact_entire_DB(DB,list_methods_name,iterations)

1 - 'market for electricity, high voltage' (kilowatt hour, IE, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Marine eutrophication, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Land transformation, biodiversity, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPA

Calculation on the entire DB --> later....

In [23]:
DB=ei33_db_no_uncert
list_methods_name=IW_with_uncert_beta_wo
iterations=1000

MC_results_eiv33_it1000_LCI0_LCIA1=MC_multi_impact_entire_DB(DB,list_methods_name,iterations)

1 - 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, CY, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Marine eutrophication, GLO, with uncert')
2 - 'electricity production, wind, 1-3MW turbine, onshore' (kilowatt hour, CN-YN, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - upda

KeyboardInterrupt: 

## Analysis of MC results

Calculating results for endpoint categories

In [77]:
#MC_results_dict={act:{ic_name:[MC_results]}} as the output of MC_multi_impact_entire_DB()

def calculating_endpoint(MC_results_dict,endpoint_name_1,endpoint_name_2):
    
    endpoint_1=[]
    endpoint_2=[]
    
    i=0
    
    for act in MC_results_dict:
        for ic_name in MC_results_dict[act]:
            
            if i==0:
                endpoint_1=[0 for j in range(len(MC_results_dict[act][ic_name]))]
                endpoint_2=[0 for j in range(len(MC_results_dict[act][ic_name]))]
                i=i+1
            
            if endpoint_name_1 in str(ic_name):
                endpoint_1=[x+y for x,y in zip(endpoint_1,MC_results_dict[act][ic_name])]
                
            
            if endpoint_name_2 in str(ic_name):
                endpoint_2=[x+y for x,y in zip(endpoint_2,MC_results_dict[act][ic_name])]
                
        MC_results_dict[act][(ic_name[0],endpoint_name_1,endpoint_name_1)]=endpoint_1
        MC_results_dict[act][(ic_name[0],endpoint_name_2,endpoint_name_2)]=endpoint_2
        
    return MC_results_dict;
                
                

Calculating statistics based on MC results

In [183]:
#MC_results_dict={act_key:{ic_name:[MC_results]}} as the output of MC_multi_impact_entire_DB()

def calculating_stats_endpoint(MC_results_dict,endpoint_name_1,endpoint_name_2):
    
    MC_results_stats={}
    
    for act in MC_results_dict:
        
        MC_results_stats[act]={}
        
        sum_spear_corr_endpoint_1=0
        sum_spear_corr_endpoint_2=0
        
        #Calculation stats
        for ic_name in MC_results_dict[act]:
            
            MC_results_stats[act][ic_name]={}
            
            #Regular stats
            MC_results_stats[act][ic_name]['mean']=np.mean(MC_results_dict[act][ic_name])
            MC_results_stats[act][ic_name]['variance']=np.var(MC_results_dict[act][ic_name])
            MC_results_stats[act][ic_name]['std dev']=np.std(MC_results_dict[act][ic_name])
            MC_results_stats[act][ic_name]['minimum']=min(MC_results_dict[act][ic_name])
            MC_results_stats[act][ic_name]['maximum']=max(MC_results_dict[act][ic_name])
            MC_results_stats[act][ic_name]['2.5th percentile']=np.percentile(MC_results_dict[act][ic_name],2.5)
            MC_results_stats[act][ic_name]['25th percentile']=np.percentile(MC_results_dict[act][ic_name],25)
            MC_results_stats[act][ic_name]['median']=np.percentile(MC_results_dict[act][ic_name],50)
            MC_results_stats[act][ic_name]['75th percentile']=np.percentile(MC_results_dict[act][ic_name],75)
            MC_results_stats[act][ic_name]['97.5th percentile']=np.percentile(MC_results_dict[act][ic_name],97.5)
            MC_results_stats[act][ic_name]['number of iterations']=len(MC_results_dict[act][ic_name])
            
            #Stats to measure the dispersion
            MC_results_stats[act][ic_name]['MADM']=np.percentile(abs(MC_results_dict[act][ic_name]-MC_results_stats[act][ic_name]['median']),50)
            MC_results_stats[act][ic_name]['IQR']=MC_results_stats[act][ic_name]['75th percentile']-MC_results_stats[act][ic_name]['25th percentile']
            MC_results_stats[act][ic_name]['Spread']=MC_results_stats[act][ic_name]['maximum']-MC_results_stats[act][ic_name]['minimum']
            MC_results_stats[act][ic_name]['CI95']=MC_results_stats[act][ic_name]['97.5th percentile']-MC_results_stats[act][ic_name]['2.5th percentile']
            MC_results_stats[act][ic_name]['Quartile coeff of dispersion']=MC_results_stats[act][ic_name]['IQR']/(MC_results_stats[act][ic_name]['75th percentile']+MC_results_stats[act][ic_name]['25th percentile'])
            MC_results_stats[act][ic_name]['CV']=MC_results_stats[act][ic_name]['std dev']/MC_results_stats[act][ic_name]['mean']
            MC_results_stats[act][ic_name]['CV modified']=MC_results_stats[act][ic_name]['std dev']
            /np.sqrt((MC_results_stats[act][ic_name]['maximum']-MC_results_stats[act][ic_name]['mean'])
                     *(MC_results_stats[act][ic_name]['mean']-MC_results_stats[act][ic_name]['minimum']))
            MC_results_stats[act][ic_name]['CV robust']=MC_results_stats[act][ic_name]['MADM']/MC_results_stats[act][ic_name]['median']
            MC_results_stats[act][ic_name]['IQR/spread']=MC_results_stats[act][ic_name]['IQR']/(MC_results_stats[act][ic_name]['Spread'])
            MC_results_stats[act][ic_name]['IQR/CI95']=MC_results_stats[act][ic_name]['IQR']/MC_results_stats[act][ic_name]['CI95']
            
            
            
            if endpoint_name_1 in str(ic_name) and endpoint_name_1 not in str(ic_name[2]):
                MC_results_stats[act][ic_name]['Spearmann rank correlation']=(sp.stats.spearmanr(MC_results_dict[act][ic_name],MC_results_dict[act][(ic_name[0],endpoint_name_1,endpoint_name_1)])[0],sp.stats.spearmanr(MC_results_dict[act][ic_name],MC_results_dict[act][(ic_name[0],endpoint_name_1,endpoint_name_1)])[1])
                sum_spear_corr_endpoint_1=sum_spear_corr_endpoint_1+(MC_results_stats[act][ic_name]['Spearmann rank correlation'][0])**2
            
            if endpoint_name_2 in str(ic_name) and endpoint_name_2 not in str(ic_name[2]):
                MC_results_stats[act][ic_name]['Spearmann rank correlation']=(sp.stats.spearmanr(MC_results_dict[act][ic_name],MC_results_dict[act][(ic_name[0],endpoint_name_2,endpoint_name_2)])[0],sp.stats.spearmanr(MC_results_dict[act][ic_name],MC_results_dict[act][(ic_name[0],endpoint_name_2,endpoint_name_2)])[1])
                sum_spear_corr_endpoint_2=sum_spear_corr_endpoint_2+(MC_results_stats[act][ic_name]['Spearmann rank correlation'][0])**2
        
        #calculating Contribution To Variance
        for ic_name in MC_results_dict[act]:
            
            if endpoint_name_1 in str(ic_name) and endpoint_name_1 not in str(ic_name[2]):
                MC_results_stats[act][ic_name]['Spearmann CTV midpoint to endpoint']=(MC_results_stats[act][ic_name]['Spearmann rank correlation'][0])**2/sum_spear_corr_endpoint_1
            
            if endpoint_name_2 in str(ic_name) and endpoint_name_2 not in str(ic_name[2]):
                MC_results_stats[act][ic_name]['Spearmann CTV midpoint to endpoint']=(MC_results_stats[act][ic_name]['Spearmann rank correlation'][0])**2/sum_spear_corr_endpoint_2
                
        
    return MC_results_stats;

In [145]:
SimplifiedDB_MC_results_dict={'LCI0 LCIA1':MC_results_eiv33_simplified_it1000_LCI0_LCIA1}
#SimplifiedDB_MC_results_dict={'LCI1 LCIA0':MC_results_eiv33_simplified_it1000_LCI1_LCIA0,'LCI0 LCIA1':MC_results_eiv33_simplified_it1000_LCI0_LCIA1}
endpoint_name_1='Human Health'
endpoint_name_2='Ecosystem Quality'

SimplifiedDB_MC_results_stats_dict={}

for uncert_setting in SimplifiedDB_MC_results_dict:
    
    MC_results_dict=SimplifiedDB_MC_results_dict[uncert_setting]
    
    MC_results_dict=calculating_endpoint(MC_results_dict,endpoint_name_1,endpoint_name_2)
    MC_results_stats=calculating_stats_endpoint(MC_results_dict,endpoint_name_1,endpoint_name_2)
    
    SimplifiedDB_MC_results_stats_dict[uncert_setting]=MC_results_stats


In [146]:
SimplifiedDB_MC_results_stats_dict

{'LCI0 LCIA1': {('ecoinvent 3.3 cutoff without uncertainty',
   '2f7d99744502bb7b4787c126d92e8569'): {('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017',
    'Ecosystem Quality',
    'Ecosystem Quality'): {'2.5th percentile': 70676.397396689412,
    '25th percentile': 118632.9827250527,
    '75th percentile': 202287.74973231557,
    '97.5th percentile': 321193.11067760532,
    'CI95': 250516.71328091592,
    'CV': 0.39640788728537002,
    'CV modified': 66519.800409165589,
    'CV robust': 0.26672250209870557,
    'IQR': 83654.767007262868,
    'IQR/CI95': 0.33392888606780069,
    'IQR/spread': 0.17586354832800816,
    'MADM': 42206.82279517244,
    'Quartile coeff of dispersion': 0.26067112076772953,
    'Spread': 475679.96780797327,
    'maximum': 521870.8175108005,
    'mean': 167806.4502315986,
    'median': 158242.45222307125,
    'minimum': 46190.8497028272,
    'number of iterations': 1000,
    'std dev': 66519.8004

## Saving Python object with pickle

In [156]:
def pickle_save(file_path_root,object_name_to_save,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    pickle.dump( object_name_to_save, open( complete_file_path, "wb" ) )

In [157]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=MC_results_eiv33_simplified_it1000_LCI0_LCIA1
file_name='MC_results_eiv33_simplified_it1000_LCI0_LCIA1'

pickle_save(file_path_root,object_name_to_save,file_name)

## Loading Python object with pickle

In [158]:
def pickle_load(file_path_root,file_name):
    
    complete_file_path=file_path_root+'\\'+file_name+'.p'
    
    return pickle.load( open( complete_file_path, "rb" ) );

In [159]:
file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'
object_name_to_save=MC_results_eiv33_simplified_it1000_LCI0_LCIA1
file_name='MC_results_eiv33_simplified_it1000_LCI0_LCIA1'

MC_results_eiv33_simplified_it1000_LCI0_LCIA1_loadtest=pickle_load(file_path_root,file_name)

# Test on a simplified DB

To create random DB from different version of ecoinvent

In [170]:
def create_simplified_random_db(complete_db,number_of_act):
    simplified_db=[complete_db.random() for i in range(10)]
    return simplified_db;

def create_db_from_db_with_same_act_code(complete_db,db_to_copy):
    act_codes=[act['code'] for act in db_to_copy]
    new_db=[act for act in complete_db if True in [act['code']==act_code for act_code in act_codes]]
    return new_db;
    

In [185]:
complete_db=ei33_db_no_uncert
number_of_act=10

ei33_db_LCI0_simplified_UP10=create_simplified_random_db(complete_db,number_of_act)

db_to_copy=ei33_db_LCI0_simplified_UP10
complete_db=ei33_db
ei33_db_LCI1_simplified_UP10=create_db_from_db_with_same_act_code(complete_db,db_to_copy)

ei33_db_LCI1_simplified_UP10

['planing, beam, softwood, u=20%' (cubic meter, RoW, None),
 'packaging glass production, brown' (kilogram, CH, None),
 'electricity production, wind, <1MW turbine, onshore' (kilowatt hour, CN-GS, None),
 'market for methanol factory' (unit, GLO, None),
 'leaching of spodumene with sulfuric acid' (kilogram, GLO, None),
 'market for p-chlorophenol' (kilogram, GLO, None),
 'linseed production' (kilogram, RU, None),
 'tomato seedling production, in unheated greenhouse, for planting' (unit, ES, None),
 'infrastructure construction, for regional distribution of oil product' (unit, RoW, None),
 'electricity, high voltage, import from SK' (kilowatt hour, HU, None)]

In [186]:
db_LCI0=ei33_db_LCI0_simplified_UP10
db_LCI1=ei33_db_LCI1_simplified_UP10
list_methods_name_LCIA0=IW_no_uncert
list_methods_name_LCIA1=IW_with_uncert_beta_wo
endpoint_name_1='Human Health'
endpoint_name_2='Ecosystem Quality'
iterations=1000

file_path_root=r'C:\bw2-python\Notebook - tutorial\IW+ integration\Pickle Python object saved'

In [190]:
def mc_calculation_and_stats(db,list_methods_name,endpoint_name_1,endpoint_name_2,iterations):
    
    MC_results_dict=MC_multi_impact_entire_DB(db,list_methods_name,iterations)
    
    MC_results_dict=calculating_endpoint(MC_results_dict,endpoint_name_1,endpoint_name_2)
    MC_results_stats=calculating_stats_endpoint(MC_results_dict,endpoint_name_1,endpoint_name_2)
    
    return {'MC results':MC_results_dict,'stats':MC_results_stats}

In [191]:
results_ei33_db_simplified_UP10_LCI0_LCIA1=mc_calculation_and_stats(db_LCI0,list_methods_name_LCIA1,endpoint_name_1,endpoint_name_2,iterations)
object_name_to_save=results_ei33_db_simplified_UP10_LCI0_LCIA1
file_name='results_ei33_db_simplified_UP10_LCI0_LCIA1'
pickle_save(file_path_root,object_name_to_save,file_name)

1 - 'infrastructure construction, for regional distribution of oil product' (unit, RoW, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Marine eutrophication, GLO, with uncert')
2 - 'market for p-chlorophenol' (kilogram, GLO, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ec

Need to code a MC calculation with multi impact categories where the same LCI values are used for all IC during the same iteration --> do not run cell below, ot does not make sense as it is!

In [179]:
results_ei33_db_simplified_UP10_LCI1_LCIA0=mc_calculation_and_stats(db_LCI1,list_methods_name_LCIA0,endpoint_name_1,endpoint_name_2,iterations)
object_name_to_save=results_ei33_db_simplified_UP10_LCI1_LCIA0
file_name='results_ei33_db_simplified_UP10_LCI1_LCIA0'
pickle_save(file_path_root,object_name_to_save,file_name)

results_ei33_db_simplified_UP10_LCI1_LCIA1=mc_calculation_and_stats(db_LCI1,list_methods_name_LCIA1,endpoint_name_1,endpoint_name_2,iterations)
object_name_to_save=results_ei33_db_simplified_UP10_LCI1_LCIA1
file_name='results_ei33_db_simplified_UP10_LCI1_LCIA1'
pickle_save(file_path_root,object_name_to_save,file_name)

1 - 'uranium fuel element production, enriched 3.8%, for light water reactor' (kilogram, CN, None)
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Freshwater acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Terrestrial acidification, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Human Health', 'Water availability, human health, GLO, with uncert')
('IMPACTWorld+ - Endpoint - only with spatial variability - Four param beta integration - update august 15th 2017', 'Ecosystem Quality', 'Marine eutrophication, GLO, with uncert')
2 - 'market for natural gas, from low pressure network (<0.1 bar), at service station' (kilogram, GLO, None)
('IMPACTWorld+ - Endpoint - only with spatial variability -

KeyboardInterrupt: 